In [9]:
## DATA
import torch

N = 100
T = 12

D1 = 5
D2 = 3
D3 = 8

V1 = torch.rand(N, T, D1) #view 1
V2 = torch.rand(N, T, D2) #view 2
V3 = torch.rand(N, D3) #view 3 with different shape

n_labels = 5
labels = torch.randint(n_labels, size=(N,1))

### Data Loader for training

In [3]:
class DataViews_torch(torch.utils.data.Dataset):
    def __init__(self, input_views: dict, target):
        super(DataViews_torch,self).__init__()
        self.target = target
        self.view_names = list(input_views.keys())
        self.views = [input_views[v_name] for v_name in self.view_names]
        
    def __len__(self):
        return len(self.views[0])
        
    def __getitem__(self, index):
        target = self.target[index]
        views = {view_n: view[index] for view, view_n in zip(self.views,self.view_names)}
        
        return {"index": index, "views": views, "target": target}

train_dataset = DataViews_torch(input_views={"view 1": V1, "view 2": V2, "view 3": V3}, target=labels)

train_dataloader = torch.utils.data.DataLoader(train_dataset,batch_size=32,drop_last=False,shuffle=True)

train_dataset[2]

{'index': 2,
 'views': {'view 1': tensor([[0.3920, 0.1860, 0.6681, 0.6972, 0.1608],
          [0.8198, 0.2070, 0.5675, 0.1441, 0.2154],
          [0.5397, 0.4983, 0.9102, 0.3904, 0.0326],
          [0.4190, 0.0594, 0.8682, 0.2950, 0.0846],
          [0.5797, 0.2838, 0.1169, 0.2154, 0.4877],
          [0.9642, 0.2956, 0.0436, 0.2391, 0.8088],
          [0.8173, 0.6105, 0.8034, 0.3139, 0.9064],
          [0.8946, 0.9759, 0.0189, 0.4850, 0.1356],
          [0.1933, 0.4255, 0.9371, 0.1092, 0.1856],
          [0.0900, 0.6817, 0.9448, 0.8154, 0.9490],
          [0.5480, 0.7739, 0.4531, 0.2991, 0.5934],
          [0.4594, 0.9377, 0.8603, 0.6569, 0.8880]]),
  'view 2': tensor([[0.7161, 0.4930, 0.9192],
          [0.6086, 0.9001, 0.5389],
          [0.1140, 0.5034, 0.3435],
          [0.9101, 0.6852, 0.9498],
          [0.1671, 0.8259, 0.6970],
          [0.2079, 0.5667, 0.0768],
          [0.8759, 0.7581, 0.0095],
          [0.6717, 0.7768, 0.9993],
          [0.2260, 0.4634, 0.7698],
        

### Model encoder, prediction head and merge function

In [15]:
### MODEL
from mvlearning.single.models import create_model
from mvlearning.merge_module import MergeModule
from mvlearning.utils import get_dic_emb_dims
from mvlearning.fusion import FeatureFusion

DIM_EMBEDDING = 32

encoder_models = {}
for name, inp_dim in {"view 1": D1, "view 2":D2}.items():
    encoder_models[name] = create_model(inp_dim, DIM_EMBEDDING, model_type="lstm")
encoder_models["view 3"] = create_model(D3, DIM_EMBEDDING, model_type="mlp") #different model architecture

EMBEDDING_DIC= get_dic_emb_dims(encoder_models)
merge_function = MergeModule(EMBEDDING_DIC, mode="concat")

prediction_head = create_model(sum(EMBEDDING_DIC.values()), n_labels, model_type="mlp", encoder=False)

# Pure Pytorch

In [11]:
## MODEL DEFINITION
mv_model = FeatureFusion(encoder_models, merge_function, prediction_head)

output_ = mv_model( {"view 1":V1, "view 2": V2, "view 3": V3})

print("Accuracy", torch.mean(((output_["prediction"] > 0.5) == labels).to(torch.float32)))

Accuracy tensor(0.1100)


In [12]:
## LOSS FUNCTION AND OPTIMIZER
optimizer = torch.optim.Adam(mv_model.parameters())
loss_function=torch.nn.CrossEntropyLoss()

loss_function(output_["prediction"], torch.squeeze(labels))

tensor(1.6202, grad_fn=<NllLossBackward0>)

In [13]:
EPOCHS = 50

for epoch in range(EPOCHS):
    
    mv_model.train(True)
    running_loss = 0.
    for i, data in enumerate(train_dataloader):
        optimizer.zero_grad()

        outputs = mv_model(data["views"])
        loss = loss_function(outputs["prediction"], torch.squeeze(data["target"]))
        loss.backward()

        optimizer.step()
        running_loss += loss
    print('EPOCH {}: with LOSS {}:'.format(epoch + 1, running_loss/len(train_dataloader)))

    ## VALIDATION/EARLY STOPPING IF REQUIRED

EPOCH 1: with LOSS 1.5883548259735107:
EPOCH 2: with LOSS 1.5919699668884277:
EPOCH 3: with LOSS 1.5976487398147583:
EPOCH 4: with LOSS 1.5683188438415527:
EPOCH 5: with LOSS 1.570216417312622:
EPOCH 6: with LOSS 1.5833370685577393:
EPOCH 7: with LOSS 1.5634428262710571:
EPOCH 8: with LOSS 1.5389440059661865:
EPOCH 9: with LOSS 1.5715234279632568:
EPOCH 10: with LOSS 1.5578805208206177:
EPOCH 11: with LOSS 1.5528523921966553:
EPOCH 12: with LOSS 1.5783663988113403:
EPOCH 13: with LOSS 1.5555310249328613:
EPOCH 14: with LOSS 1.551052451133728:
EPOCH 15: with LOSS 1.5096633434295654:
EPOCH 16: with LOSS 1.5071043968200684:
EPOCH 17: with LOSS 1.5813878774642944:
EPOCH 18: with LOSS 1.5382052659988403:
EPOCH 19: with LOSS 1.5652226209640503:
EPOCH 20: with LOSS 1.5213960409164429:
EPOCH 21: with LOSS 1.5262451171875:
EPOCH 22: with LOSS 1.5798885822296143:
EPOCH 23: with LOSS 1.500242829322815:
EPOCH 24: with LOSS 1.5035268068313599:
EPOCH 25: with LOSS 1.5111453533172607:
EPOCH 26: with 

In [14]:
output_ = mv_model( {"view 1":V1, "view 2": V2, "view 3": V3})

print("Accuracy", torch.mean(((output_["prediction"] > 0.5) == labels).to(torch.float32)))

Accuracy tensor(0.1360)


# Using Pytorch Lightning

In [16]:
import pytorch_lightning as pl

## MODEL DEFINITION with LOSS FUNCTION and OPTIMIZER INSIDE
mv_model = FeatureFusion(encoder_models, merge_function, prediction_head, 
                         loss_function=torch.nn.CrossEntropyLoss(), optimizer="adam")

mv_model.configure_optimizers()

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.001
    maximize: False
    weight_decay: 0
)

In [17]:
mv_model.loss_batch({
    "views": {"view 1":V1, "view 2": V2, "view 3": V3},
    "target": labels
})

{'objective': tensor(1.6020, grad_fn=<NllLossBackward0>)}

In [18]:
output_ = mv_model( {"view 1":V1, "view 2": V2, "view 3": V3})

print("Accuracy", torch.mean(((output_["prediction"] > 0.5) == labels).to(torch.float32)))

Accuracy tensor(0.1100)


### Train

In [19]:
trainer = pl.Trainer(max_epochs=50, accelerator="gpu", devices = 1)
trainer.fit(mv_model, train_dataloader, val_dataloaders=None)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/panshop/anaconda3/envs/views/lib/python3.8/site-packages/pytorch_lightning/trainer/configuration_validator.py:133: UserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn("You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name            | Type             | Params
-----------------------------------------------------
0 | views_encoder   | ModuleDict       | 431 K 
1 | merge_module    | MergeModule      | 0     
2 | prediction_head | Generic_Decoder  | 29.6 K
3 | loss_function   | CrossEntropyLoss | 0     
-----------------------------------------------------
461 K     Trainable params
0         Non-trainable params
461 K     Total params
1.844     Total estimated model params size (MB)
/home/panshop/anac

Training: 0it [00:00, ?it/s]

In [21]:
output_ = mv_model( {"view 1":V1, "view 2": V2, "view 3": V3})

print("Accuracy", torch.mean(((output_["prediction"] > 0.5) == labels).to(torch.float32)))

Accuracy tensor(0.1400)
